#### Prediction Model to predict the XY coordinate of the staff

In [1]:
import os
import numpy as np
import cv2 # Extract images from video
import tensorflow as tf # Tensorflow stuff
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import pandas as pd

#### Training Data

In [2]:
df = pd.read_csv('./data/xy_locate/train.csv')
df.head()
X_img_name = df['img_number']
X = []
y = df[['x_position', 'y_position']]

for i in range(len(X_img_name)):
    im = cv2.imread('./data/xy_locate/{}'.format(str(X_img_name[i])))
    X.append(im)

X = np.array(X)
y = np.array(y)

print("x: " + str(X.shape) + "  y: " + str(y.shape))

x: (20, 720, 960, 3)  y: (20, 2)


In [3]:
# Resize, normalising X and preprocess for ResNet50
X = X.astype('float32')
X /= 255

X = tf.image.resize(images=X, size=[240, 320])
X = tf.keras.applications.resnet50.preprocess_input(X)
X = X.numpy()

# Split x and y into training and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)

#### Model Preparation

In [4]:
# USE WEIGHTS FROM PREDICTION MODEL
# base_model = tf.keras.applications.ResNet50(input_shape=(240, 320, 3), include_top=False, weights='imagenet')
loaded_model = tf.keras.models.load_model("./model/prediction/prediction_model_2")

# Freeze trainable layers except for last few layers
loaded_model.trainable = False

output = loaded_model.layers[-6].output
output = tf.keras.layers.Flatten() (output)

# Add new FC Layers to the output that can predict the XY coordinate and prediction
output = tf.keras.layers.Dense(128, activation="relu") (output)
output = tf.keras.layers.Dense(64, activation="relu") (output)
output = tf.keras.layers.Dense(32, activation="relu") (output)
output = tf.keras.layers.Dense(2, activation=None) (output) # no need activation function, 2 units for XY coordinates of the tag

model = tf.keras.models.Model(inputs=loaded_model.input, outputs=output)
# model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 240, 320, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 246, 326, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 120, 160, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [9]:
# MODEL PARAMETERS
EPOCHS = 50
BATCH_SIZE = 4
OPTIMIZER = tf.optimizers.Adam()
LOSS_FUNC = tf.keras.losses.mean_squared_error # want to look at the error between actual and prediction

CALLBACKS = [
    tf.keras.callbacks.ModelCheckpoint(filepath="./model/xy_locate/xy_locate_model", monitor='accuracy', mode='max', verbose=False, save_best_only=True)
        ]

# Initialize optimizer, compile and start fine tuning the model
model.compile(optimizer=OPTIMIZER, loss=LOSS_FUNC, metrics=['accuracy'])

# Fit the model
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, callbacks=CALLBACKS, epochs=EPOCHS, verbose=1, validation_data=(X_test, y_test))

Epoch 1/200
1/4 [======>.......................] - ETA: 6s - loss: 43668.1641 - accuracy: 0.5000

INFO:tensorflow:Assets written to: ./model/xy_locate\xy_locate_model\assets


INFO:tensorflow:Assets written to: ./model/xy_locate\xy_locate_model\assets


4/4 [==============================] - 17s 5s/step - loss: 44829.6641 - accuracy: 0.5000 - val_loss: 20975.0234 - val_accuracy: 0.5000
Epoch 2/200
4/4 [==============================] - 0s 25ms/step - loss: 42243.8398 - accuracy: 0.5000 - val_loss: 26835.5039 - val_accuracy: 0.5000
Epoch 3/200
4/4 [==============================] - 0s 25ms/step - loss: 43412.8906 - accuracy: 0.5000 - val_loss: 23511.8438 - val_accuracy: 0.5000
Epoch 4/200
4/4 [==============================] - 0s 24ms/step - loss: 42202.1953 - accuracy: 0.5000 - val_loss: 19869.1406 - val_accuracy: 0.5000
Epoch 5/200
4/4 [==============================] - 0s 25ms/step - loss: 40477.4805 - accuracy: 0.5000 - val_loss: 20942.8242 - val_accuracy: 0.5000
Epoch 6/200
4/4 [==============================] - 0s 24ms/step - loss: 41569.1016 - accuracy: 0.5000 - val_loss: 23842.3281 - val_accuracy: 0.5000
Epoch 7/200
4/4 [==============================] - 0s 24ms/step - loss: 40945.4531 - accuracy: 0.5000 - val_loss: 20931.2402 

INFO:tensorflow:Assets written to: ./model/xy_locate\xy_locate_model\assets


...

INFO:tensorflow:Assets written to: ./model/xy_locate\xy_locate_model\assets


4/4 [==============================] - 14s 5s/step - loss: 21996.7266 - accuracy: 0.9375 - val_loss: 14641.8770 - val_accuracy: 0.5000
Epoch 45/200
4/4 [==============================] - 0s 25ms/step - loss: 17087.4141 - accuracy: 0.6875 - val_loss: 20914.6348 - val_accuracy: 0.5000
Epoch 46/200
4/4 [==============================] - 0s 26ms/step - loss: 18662.9434 - accuracy: 0.5625 - val_loss: 10316.7842 - val_accuracy: 0.5000
Epoch 47/200
4/4 [==============================] - 0s 25ms/step - loss: 15250.1758 - accuracy: 0.8750 - val_loss: 13510.9814 - val_accuracy: 0.5000
Epoch 48/200
4/4 [==============================] - 0s 25ms/step - loss: 15640.3271 - accuracy: 0.8750 - val_loss: 14482.3105 - val_accuracy: 0.5000
Epoch 49/200
4/4 [==============================] - 0s 27ms/step - loss: 15763.0449 - accuracy: 0.8125 - val_loss: 9498.5547 - val_accuracy: 0.5000
Epoch 50/200
4/4 [==============================] - 0s 25ms/step - loss: 16470.5059 - accuracy: 0.6250 - val_loss: 19182.

INFO:tensorflow:Assets written to: ./model/xy_locate\xy_locate_model\assets


INFO:tensorflow:Assets written to: ./model/xy_locate\xy_locate_model\assets


4/4 [==============================] - 14s 5s/step - loss: 13673.1924 - accuracy: 1.0000 - val_loss: 20351.0234 - val_accuracy: 0.5000
Epoch 55/200
4/4 [==============================] - 0s 25ms/step - loss: 13302.6240 - accuracy: 0.8125 - val_loss: 15053.9668 - val_accuracy: 0.5000
Epoch 56/200
4/4 [==============================] - 0s 25ms/step - loss: 6720.2261 - accuracy: 0.9375 - val_loss: 7568.9390 - val_accuracy: 0.7500
Epoch 57/200
4/4 [==============================] - 0s 24ms/step - loss: 8432.9756 - accuracy: 1.0000 - val_loss: 15068.2441 - val_accuracy: 0.5000
Epoch 58/200
4/4 [==============================] - 0s 25ms/step - loss: 7612.2100 - accuracy: 0.8750 - val_loss: 14811.3447 - val_accuracy: 0.5000
Epoch 59/200
4/4 [==============================] - 0s 25ms/step - loss: 8335.8242 - accuracy: 1.0000 - val_loss: 9731.9141 - val_accuracy: 0.5000
Epoch 60/200
4/4 [==============================] - 0s 25ms/step - loss: 8875.0117 - accuracy: 0.8125 - val_loss: 13972.1836 -

In [6]:
# Save Model
model.save("./model/xy_locate/xy_locate_model_2")